# Laboratorio 1

---

## Ejercicio 1: Optimización de Hiperparámetros para Máxima Precisión

1. Experimenta con los siguientes hiperparámetros para maximizar la precisión en el conjunto de prueba:
   - **Optimizador**: Prueba `adam`, `sgd`.
   - **Batch size**: Experimenta con `32`, `64`, `128`.
   - **Épocas**: Prueba con `10`, `20`.
   - **Neuronas y capas**: Varía el número de neuronas y capas densas.


   - Documenta los hiperparámetros probados y sus resultados.
   - Presenta los resultados en una tabla para analizar patrones.
   - Encuentre el mejor modelo. Llamaremos este modelo MMM

---

In [4]:
# Desactiva las advertencias para evitar saturar la salida con mensajes irrelevantes
import warnings
warnings.filterwarnings('ignore')

# Verifica la versión de TensorFlow
import tensorflow as tf

# Módulos de TensorFlow y Keras para redes neuronales
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

# Bibliotecas para manipulación de datos y cálculos numéricos
import numpy as np
import pandas as pd

# Librerías para visualización de datos
import matplotlib.pyplot as plt

# Módulos de evaluación de modelos
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score
)

# Módulos auxiliares
import sys  # Para interactuar con el sistema operativo
import os
import pickle  # Para guardar y cargar modelos o datos serializados
from itertools import product  # Para generar combinaciones de valores

# Habilita la visualización de gráficos en línea (solo en Jupyter Notebook)
%matplotlib inline


In [2]:
# Carga el conjunto de datos MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [7]:
# Crear carpeta para guardar modelos
save_dir = "modelos_guardados"
os.makedirs(save_dir, exist_ok=True)

i = 1

# Hiperparámetros a explorar
optimizers_list = ['adam', 'sgd']
batch_sizes = [32, 64, 128]
epochs_list = [10, 20]
num_neurons_list = [50, 100]
num_layers_list = [5, 10]

# Cargar datos de MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape((x_train.shape[0], 28 * 28)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], 28 * 28)).astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Almacenar resultados
results = []


# Iterar sobre combinaciones de hiperparámetros
for optimizer, batch_size, epochs, num_layers, num_neurons in product(
    optimizers_list, batch_sizes, epochs_list, num_layers_list, num_neurons_list):

    print(f"Entrenando modelo con: Opt={optimizer}, Batch={batch_size}, Epochs={epochs}, Layers={num_layers}, Neurons={num_neurons}")

    # Construir modelo
    model = models.Sequential()
    model.add(layers.Dense(num_neurons, activation='relu', input_shape=(28*28,)))
    for _ in range(num_layers - 1):
        model.add(layers.Dense(num_neurons, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    # Compilar modelo
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy', 'Recall', 'Precision', 'AUC'])

    # Entrenar modelo
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)

    # Obtener métricas de la última época (corregido: usar siempre la última)
    best_epoch = epochs  # Se usa el número de épocas directamente
    best_acc = history.history['val_accuracy'][-1]
    best_precision = history.history.get('val_precision', [0] * epochs)[-1]
    best_recall = history.history.get('val_recall', [0] * epochs)[-1]
    best_f1 = 2 * (best_precision * best_recall) / (best_precision + best_recall + 1e-7)
    best_auc = history.history.get('val_auc', [0] * epochs)[-1]

    # Evaluar modelo en prueba
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    test_acc = accuracy_score(y_test_classes, y_pred_classes)
    test_precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
    test_recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
    test_f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')
    test_auc = roc_auc_score(y_test, y_pred, multi_class='ovr', average='weighted')

    # Nombre y ruta del modelo
    model_name = f"model_{i}.h5"
    i += 1
    model_path = os.path.join(save_dir, model_name)
    
    # Guardar el modelo
    model.save(model_path)

    # Guardar resultados
    results.append({
        'optimizer': optimizer,
        'batch_size': batch_size,
        'epochs': best_epoch,
        'num_layers': num_layers,
        'num_neurons': num_neurons,
        'best_val_accuracy': best_acc,
        'best_val_precision': best_precision,
        'best_val_recall': best_recall,
        'best_val_f1_score': best_f1,
        'best_val_auc': best_auc,
        'test_accuracy': test_acc,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_f1_score': test_f1,
        'test_auc': test_auc,
        'model_name': model_name,
        'model_path': model_path
    })

# Guardar resultados en un DataFrame
results_df = pd.DataFrame(results)
results_df.to_csv('model_results.csv', index=False)

print("Modelos guardados en:", save_dir)
print("Resultados guardados en: model_results.csv")

Entrenando modelo con: Opt=adam, Batch=32, Epochs=10, Layers=5, Neurons=50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Entrenando modelo con: Opt=adam, Batch=32, Epochs=10, Layers=5, Neurons=100
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Entrenando modelo con: Opt=adam, Batch=32, Epochs=10, Layers=10, Neurons=50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Entrenando modelo con: Opt=adam, Batch=32, Epochs=10, Layers=10, Neurons=100
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Entrenando modelo con: Opt=adam, Batch=32, Epochs=20, Layers=5, Neurons=50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


...

Entrenando modelo con: Opt=sgd, Batch=128, Epochs=20, Layers=5, Neurons=50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Entrenando modelo con: Opt=sgd, Batch=128, Epochs=20, Layers=5, Neurons=100
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Entrenando modelo con: Opt=sgd, Batch=128, Epochs=20, Layers=10, Neurons=50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Entrenando modelo con: Opt=sgd, Batch=128, Epochs=20, Layers=10, Neurons=100
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Modelos guardados en: modelos_guardados
Resultados guardados en: model_results.csv


In [3]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix

In [45]:
import os
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.figure_factory as ff
import plotly.graph_objects as go
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

# Cargar datos de MNIST
(x_train, y_train_labels), (x_test, y_test_labels) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape((x_train.shape[0], 28 * 28)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], 28 * 28)).astype('float32') / 255
y_train = to_categorical(y_train_labels)
y_test = to_categorical(y_test_labels)

# Crear carpeta para las matrices de confusión
conf_matrix_dir = "confusion_matrices"
os.makedirs(conf_matrix_dir, exist_ok=True)

# Cargar CSV
csv_path = "model_results.csv"
df = pd.read_csv(csv_path,delimiter=";")

# Procesar cada modelo
def process_model(model_path):
    model = load_model(model_path)
    
    # Predicciones
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    
    y_pred_classes_train = np.argmax(y_pred_train, axis=1)
    y_pred_classes_test = np.argmax(y_pred_test, axis=1)
    
    # Reportes de clasificación
    target_names = [f"Clase {i}" for i in range(10)]
    train_report = classification_report(y_train_labels, y_pred_classes_train, target_names=target_names, digits=4)
    test_report = classification_report(y_test_labels, y_pred_classes_test, target_names=target_names, digits=4)
    
    # Matriz de confusión
    conf_matrix = confusion_matrix(y_test_labels, y_pred_classes_test)
    colorscale = [[0, "#EFEFEF"], [1, "#CC3F0C"]]
    fig = go.Figure(data=go.Heatmap(z=conf_matrix, colorscale=colorscale, x=target_names, y=target_names, showscale=False))
    
    # Añadir los valores en cada celda
    for i in range(len(target_names)):
        for j in range(len(target_names)):
            fig.add_annotation(text=str(conf_matrix[i, j]), x=target_names[j], y=target_names[i], showarrow=False, font=dict(color="black"))
    
    fig.update_layout(title="Matriz de Confusión",
                      xaxis_title="Etiquetas Predichas",
                      yaxis_title="Etiquetas Reales")
    
    # Guardar la figura como .pkl
    conf_matrix_path = os.path.join(conf_matrix_dir, os.path.basename(model_path) + ".pkl")
    with open(conf_matrix_path, "wb") as f:
        pickle.dump(fig, f)
    
    return train_report, test_report, conf_matrix_path

# Aplicar a cada modelo
df[['train_report', 'test_report', 'conf_matrix_path']] = df['model_path'].apply(lambda path: pd.Series(process_model(path)))

# Guardar el CSV actualizado
df.to_csv(csv_path, index=False)
print("Proceso completado y CSV actualizado.")


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


...

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Proceso completado y CSV actualizado.
